In [2]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.3 MB/s eta 0:00:00


In [11]:
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import os

# Merkle Tree Functions
def hash_chunk(chunk):
    return hashlib.sha256(chunk).hexdigest()

def build_tree(chunks):
    hashes = [hash_chunk(chunk) for chunk in chunks]
    tree = [hashes]
    while len(hashes) > 1:
        if len(hashes) % 2 != 0:
            hashes.append(hashes[-1])  # Duplicate last hash if odd
        hashes = [hash_chunk((hashes[i] + hashes[i + 1]).encode()) for i in range(0, len(hashes), 2)]
        tree.append(hashes)
    return tree

def get_merkle_root(chunks):
    tree = build_tree(chunks)
    return tree[-1][0] if tree else None

def print_merkle_tree(chunks):
    tree = build_tree(chunks)
    for level, hashes in enumerate(tree):
        print(f"Level {level}:")
        for i, h in enumerate(hashes):
            print(f"  {i}. {h}")

# Encryption and Decryption Functions
KEY = os.urandom(16)
IV = os.urandom(16)

def encrypt_chunk(chunk):
    cipher = AES.new(KEY, AES.MODE_CBC, IV)
    return cipher.encrypt(pad(chunk, AES.block_size))

def decrypt_chunk(chunk):
    cipher = AES.new(KEY, AES.MODE_CBC, IV)
    return unpad(cipher.decrypt(chunk), AES.block_size)

# Function to Read and Split File into Chunks
def read_file_chunks(file_path, chunk_size=1024):
    chunks = []
    with open(file_path, 'rb') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            chunks.append(chunk)
    return chunks

# Example Usage
if __name__ == '__main__':
    file_path = 'file.txt'
    file_chunks = read_file_chunks(file_path)
    encrypted_chunks = [encrypt_chunk(chunk) for chunk in file_chunks]
    print("Merkle Root:", get_merkle_root(file_chunks))
    print("\nMerkle Tree:")
    print_merkle_tree(file_chunks)
    decrypted_chunks = [decrypt_chunk(chunk) for chunk in encrypted_chunks]
    print("\nDecrypted Chunks:")
    for i, chunk in enumerate(decrypted_chunks, 1):
        print(f"Chunk {i}:\n{chunk.decode(errors='replace')}\n")


Merkle Root: a19a5fadab5b36024dd7a948e67ac53a36993ec5f2d1ee6cc2ba014407891f55

Merkle Tree:
Level 0:
  0. 024f5068587ef14149e349c222a1346c82f33096e635c3e1889b217ff0461c44
  1. 7114df8793e6b0b60f478b6c5e2ce26a895f4d29204c9414fc1b7144de7f7153
  2. 7f32b846542753a6cc27e8ddecf9bd5f83ebdfd6364392183734e83b97a74f28
  3. aededb810b4ee28a8071a4e15d0c3128415c243894731d189c4de7d1f9344b0a
  4. ef711319910f7de4d977253204753bcadf77007842169d1b56f2adadc8f5cafd
  5. ef711319910f7de4d977253204753bcadf77007842169d1b56f2adadc8f5cafd
Level 1:
  0. 27bfc21787eac73fa97d1b2700438fd37c30ae6fe0a4e4a9b0b659d1c0900c28
  1. ad58f2b1e7fa3d1fd950556b997893794437f066b5966ff0e09f09f4819a9018
  2. 1b9b7f850cc5edd911e0e22774b74a2fae3132d9eaa1932d404ff1d489a8a77c
  3. 1b9b7f850cc5edd911e0e22774b74a2fae3132d9eaa1932d404ff1d489a8a77c
Level 2:
  0. fa60aa7ed4ba923da5b5e7198b60fbf44bfa5106acc9c1563e0bcb5dc8aca122
  1. 30b804cb5c0337e0110a39ff8df5488767d9c391c9be1333e7fc3ceb2ef05dd5
Level 3:
  0. a19a5fadab5b36024dd7a948e67